#### Creating Custom Foreign Conditions

Another element of the _primary join condition_ is how those columns considered __`"foreign"`__ are determined. Usually, some _subset of Column objects_ will specify _ForeignKey_, or otherwise be _part of a ForeignKeyConstraint_ that's relevant to the join condition. `relationship()` looks to this _foreign key status_ as it decides __how it should load and persist data__ for this relationship. However, the `relationship.primaryjoin` argument can be used to __create a join condition that doesn't involve any `"schema"` level foreign keys__. We can combine `relationship.primaryjoin` along with *relationship.foreign_keys* and *relationship.remote_side* __explicitly__ in order to establish such a join.

Below, a class _HostEntry_ joins to itself, equating the string content column to the ip_address column, which is a _PostgreSQL_ type called _INET_. We need to use `cast()` in order to cast one side of the join to the type of the other.

In [1]:
from sqlalchemy import cast, String, Column, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import INET
from sqlalchemy.orm import relationship, foreign, remote

In [2]:
Base = declarative_base()

In [3]:
class HostEntry(Base):
    __tablename__ = "host_entry"
    
    id = Column(Integer, primary_key=True)
    ip_address = Column(INET)
    content = Column(String(50))
    
    # relationship() using explicit foreign_keys, remote_side
    parent_host = relationship(
        "HostEntry",
        primaryjoin=ip_address == cast(content, INET),
        foreign_keys=content,
        remote_side=ip_address,
    )

The above relationship will produce a join like:

```
SELECT host_entry.id, host_entry.ip_address, host_entry.content
FROM host_entry JOIN host_entry AS host_entry_1
ON host_entry_1.ip_address = CAST(host_entry.content AS INET)
```

An alternative syntax to the above is to use the `foreign()` and `remote()` _annotations_, __inline within__ the `relationship.primaryjoin` expression. This syntax represents the _annotations_ that `relationship()` __normally applies by itself__ to the join condition given the *relationship.foreign_keys* and *relationship.remote_side* arguments. These functions may be __more succinct when an explicit join condition is present__, and additionally serve to mark exactly the column that is __"foreign"__ or __"remote"__ independent of whether that column is stated multiple times or within complex SQL expressions.

In [4]:
class SuccinctHostEntry(Base):
    __tablename__ = "succinct_host_entry"
    
    id = Column(Integer, primary_key=True)
    ip_address = Column(INET)
    content = Column(String(50))
    
    # relationship() using explicit foreign() and remote() annotations
    # in lieu of separate arguments
    parent_host = relationship(
        "SuccinctHostEntry",
        primaryjoin=remote(ip_address) == cast(foreign(content), INET),
    )